In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
print("API Key loaded:", bool(api_key))

API Key loaded: True


In [2]:
import requests
from datetime import datetime, timedelta 

# random check on data retrieval
url = "https://gateway.api.bot.or.th/Stat-ExchangeRate/v2/DAILY_AVG_EXG_RATE/" 

# the time frame of each call is 30 days
params = {
    "start_period":"2025-01-01",
    "end_period":"2025-01-31"
}

headers={
    "Authorization":f"{api_key}",
    "Accept":"application/json"
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
print(response.status_code)
print(response.text)
# nice :)

200
{"result":{"timestamp":"2025-10-21 05:24:51","api":"Daily Weighted-average Interbank Exchange Rate - THB / USD","data":{"data_header":{"report_name_eng":"Rates of Exchange of Commercial Banks in Bangkok Metropolis (2002-present)","report_name_th":"อัตราแลกเปลี่ยนเฉลี่ยของธนาคารพาณิชย์ในกรุงเทพมหานคร (2545-ปัจจุบัน)","report_uoq_name_eng":"(Unit: Baht / 1 Unit of Foreign Currency)","report_uoq_name_th":"(หน่วย: บาท ต่อ 1 หน่วยเงินตราต่างประเทศ)","report_source_of_data":[{"source_of_data_eng":"Bank of Thailand","source_of_data_th":"ธนาคารแห่งประเทศไทย"}],"report_remark":[{"report_remark_eng":"Since Nov 16, 2015 the data regarding Buying Transfer Rate of PKR has been changed to Buying Rate using Foreign Exchange Rates (LSEG) with Bangkok Market Crossing.","report_remark_th":"ตั้งแต่วันที่ 16 พ.ย. 2558 ข้อมูลในอัตราซื้อเงินโอนของสกุล PKR ได้เปลี่ยนเป็นอัตราซื้อที่ใช้อัตราในตลาดต่างประเทศ (LSEG) คำนวณผ่านอัตราซื้อขายเงินดอลลาร์ สรอ. ในตลาดกรุงเทพฯ"}],"last_updated":"2025-10-20"},"data_d

In [18]:
data

{'result': {'timestamp': '2025-10-19 00:05:56',
  'api': 'Daily Weighted-average Interbank Exchange Rate - THB / USD',
  'data': {'data_header': {'report_name_eng': 'Rates of Exchange of Commercial Banks in Bangkok Metropolis (2002-present)',
    'report_name_th': 'อัตราแลกเปลี่ยนเฉลี่ยของธนาคารพาณิชย์ในกรุงเทพมหานคร (2545-ปัจจุบัน)',
    'report_uoq_name_eng': '(Unit: Baht / 1 Unit of Foreign Currency)',
    'report_uoq_name_th': '(หน่วย: บาท ต่อ 1 หน่วยเงินตราต่างประเทศ)',
    'report_source_of_data': [{'source_of_data_eng': 'Bank of Thailand',
      'source_of_data_th': 'ธนาคารแห่งประเทศไทย'}],
    'report_remark': [{'report_remark_eng': 'Since Nov 16, 2015 the data regarding Buying Transfer Rate of PKR has been changed to Buying Rate using Foreign Exchange Rates (LSEG) with Bangkok Market Crossing.',
      'report_remark_th': 'ตั้งแต่วันที่ 16 พ.ย. 2558 ข้อมูลในอัตราซื้อเงินโอนของสกุล PKR ได้เปลี่ยนเป็นอัตราซื้อที่ใช้อัตราในตลาดต่างประเทศ (LSEG) คำนวณผ่านอัตราซื้อขายเงินดอลลาร์ สรอ

In [26]:
data['result']['data']['data_detail']

[{'period': '2025-01-31',
  'currency_id': 'USD',
  'currency_name_th': 'สหรัฐอเมริกา : ดอลลาร์ (USD)',
  'currency_name_eng': 'USA : DOLLAR (USD) ',
  'buying_sight': '33.4092000',
  'buying_transfer': '33.4853000',
  'selling': '33.8042000',
  'mid_rate': '33.6448000'},
 {'period': '2025-01-31',
  'currency_id': 'GBP',
  'currency_name_th': 'อังกฤษ : ปอนด์สเตอร์ลิง (GBP)',
  'currency_name_eng': 'UNITED KINGDOM : POUND STERING (GBP)',
  'buying_sight': '41.3458000',
  'buying_transfer': '41.4482000',
  'selling': '42.2217000',
  'mid_rate': '41.8350000'},
 {'period': '2025-01-31',
  'currency_id': 'EUR',
  'currency_name_th': 'ยูโรโซน : ยูโร (EUR)',
  'currency_name_eng': 'EURO ZONE : EURO (EUR)',
  'buying_sight': '34.6041000',
  'buying_transfer': '34.6845000',
  'selling': '35.3091000',
  'mid_rate': '34.9968000'},
 {'period': '2025-01-31',
  'currency_id': 'JPY',
  'currency_name_th': 'ญี่ปุ่น : เยน (100 เยน) (JPY)',
  'currency_name_eng': 'JAPAN : YEN (100 YEN) (JPY) ',
  'buyin

In [30]:
import pandas as pd

def fetch_exchange_rate(currency="USD", years=10, save_csv=True, filename="exchange_rate.csv"):
    headers = {"Authorization": f"Bearer {api_key}", "accept": "application/json"}

    url = "https://gateway.api.bot.or.th/Stat-ExchangeRate/v2/DAILY_AVG_EXG_RATE/"

    end_date = datetime.today()
    start_date = end_date - timedelta(days=years*365)

    current_start = start_date
    records = []

    # loop over 31 days time frame
    while current_start < end_date:
        # Calculate chunk end date
        current_end = min(current_start + timedelta(days=30), end_date)

        # Format dates for API
        start_str = current_start.strftime("%Y-%m-%d")
        end_str = current_end.strftime("%Y-%m-%d")

        print(f"Fetching {start_str} → {end_str} ...")

        # Call API with query parameters
        params = {
            "start_period": start_str,
            "end_period": end_str,
            "currency": currency
        }
        
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if "result" in data and "data" in data["result"]:
                details = data["result"]["data"]["data_detail"]
                records.extend(details)
            else:
                print("no data found")
        else:
            print(f"error {response.status_code}: {response.text}")

        # next time frame
        current_start = current_end + timedelta(days=1)

    df = pd.DataFrame(records)

    if not df.empty:
        for col in ["buying_sight", "buying_transfer", "selling", "mid_rate"]:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        df["period"] = pd.to_datetime(df["period"])
    else:
        print("no records!")
    return df

df = fetch_exchange_rate(currency="USD", years=10)

Fetching 2015-10-22 → 2015-11-21 ...
Fetching 2015-11-22 → 2015-12-22 ...
Fetching 2015-12-23 → 2016-01-22 ...
Fetching 2016-01-23 → 2016-02-22 ...
Fetching 2016-02-23 → 2016-03-24 ...
Fetching 2016-03-25 → 2016-04-24 ...
Fetching 2016-04-25 → 2016-05-25 ...
Fetching 2016-05-26 → 2016-06-25 ...
Fetching 2016-06-26 → 2016-07-26 ...
Fetching 2016-07-27 → 2016-08-26 ...
Fetching 2016-08-27 → 2016-09-26 ...
Fetching 2016-09-27 → 2016-10-27 ...
Fetching 2016-10-28 → 2016-11-27 ...
Fetching 2016-11-28 → 2016-12-28 ...
Fetching 2016-12-29 → 2017-01-28 ...
Fetching 2017-01-29 → 2017-02-28 ...
Fetching 2017-03-01 → 2017-03-31 ...
Fetching 2017-04-01 → 2017-05-01 ...
Fetching 2017-05-02 → 2017-06-01 ...
Fetching 2017-06-02 → 2017-07-02 ...
Fetching 2017-07-03 → 2017-08-02 ...
Fetching 2017-08-03 → 2017-09-02 ...
Fetching 2017-09-03 → 2017-10-03 ...
Fetching 2017-10-04 → 2017-11-03 ...
Fetching 2017-11-04 → 2017-12-04 ...
Fetching 2017-12-05 → 2018-01-04 ...
Fetching 2018-01-05 → 2018-02-04 ...
F

In [27]:
# i did save data from my very previous attempt so pls imagine that it works THANKS 
df.to_csv("../data/exchange_rate_usd", index=False)

NameError: name 'df' is not defined

In [4]:
import pandas as pd
def fetch_weighted_exchange_rate(currency="USD", years=10, save_csv=True, filename="exchange_rate.csv"):
    headers = {"Authorization": f"Bearer {api_key}", "accept": "application/json"}

    url = "https://gateway.api.bot.or.th/Stat-ReferenceRate/v2/DAILY_REF_RATE/"

    end_date = datetime.today()
    start_date = end_date - timedelta(days=years*365)

    current_start = start_date
    records = []

    # loop over 31 days time frame
    while current_start < end_date:
        # Calculate chunk end date
        current_end = min(current_start + timedelta(days=30), end_date)

        # Format dates for API
        start_str = current_start.strftime("%Y-%m-%d")
        end_str = current_end.strftime("%Y-%m-%d")

        print(f"Fetching {start_str} → {end_str} ...")

        # Call API with query parameters
        params = {
            "start_period": start_str,
            "end_period": end_str,
            "currency": currency
        }
        
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            if "result" in data and "data" in data["result"]:
                details = data["result"]["data"]["data_detail"]
                records.extend(details)
            else:
                print("no data found")
        else:
            print(f"error {response.status_code}: {response.text}")

        # next time frame
        current_start = current_end + timedelta(days=1)

    df = pd.DataFrame(records)

    if not df.empty:
        df["rate"] = pd.to_numeric(df["rate"], errors="coerce")
        df["period"] = pd.to_datetime(df["period"])
    else:
        print("no records!")
    return df

df2 = fetch_weighted_exchange_rate(currency="USD", years=10)

Fetching 2015-10-24 → 2015-11-23 ...
Fetching 2015-11-24 → 2015-12-24 ...
Fetching 2015-12-25 → 2016-01-24 ...
Fetching 2016-01-25 → 2016-02-24 ...
Fetching 2016-02-25 → 2016-03-26 ...
Fetching 2016-03-27 → 2016-04-26 ...
Fetching 2016-04-27 → 2016-05-27 ...
Fetching 2016-05-28 → 2016-06-27 ...
Fetching 2016-06-28 → 2016-07-28 ...
Fetching 2016-07-29 → 2016-08-28 ...
Fetching 2016-08-29 → 2016-09-28 ...
Fetching 2016-09-29 → 2016-10-29 ...
Fetching 2016-10-30 → 2016-11-29 ...
Fetching 2016-11-30 → 2016-12-30 ...
Fetching 2016-12-31 → 2017-01-30 ...
Fetching 2017-01-31 → 2017-03-02 ...
Fetching 2017-03-03 → 2017-04-02 ...
Fetching 2017-04-03 → 2017-05-03 ...
Fetching 2017-05-04 → 2017-06-03 ...
Fetching 2017-06-04 → 2017-07-04 ...
Fetching 2017-07-05 → 2017-08-04 ...
Fetching 2017-08-05 → 2017-09-04 ...
Fetching 2017-09-05 → 2017-10-05 ...
Fetching 2017-10-06 → 2017-11-05 ...
Fetching 2017-11-06 → 2017-12-06 ...
Fetching 2017-12-07 → 2018-01-06 ...
Fetching 2018-01-07 → 2018-02-06 ...
F

In [5]:
df2.to_csv("../data/weighted_exchange_rate_usd", index=False)

In [6]:
df2

,period,rate
0,2015-11-23,35.824
1,2015-11-20,35.713
2,2015-11-19,35.926
3,2015-11-18,36.041
4,2015-11-17,35.965
...,...,...
2424,2025-10-03,32.436
2425,2025-10-02,32.429
2426,2025-10-01,32.444
2427,2025-09-30,32.328
